# LUAD CPTAC analysis

In [1]:
import pandas as pd
import numpy as np
from msresist.pre_processing import preprocessing
from msresist.clustering import MassSpecClustering
import matplotlib.pyplot as plt

In [2]:
# pd.set_option('display.max_colwidth', 1000)
# pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_columns', 1000)

## 1 - Import and Preprocess Data

In [3]:
# X = preprocessing(CPTAC=True, log2T=True)
X = pd.read_csv("msresist/data/MS/CPTAC/CPTAC-preprocessedMotfis.csv")

d = X.select_dtypes(include=['float64']).T
i = X.select_dtypes(include=['object'])

Drop NaNs:

In [4]:
d = d.iloc[:8, :].dropna(axis=1)
i = i.iloc[d.columns, :]

Reduce data set:

In [5]:
# npept = 5000
# d = d.iloc[:, :npept]
# i = i.iloc[:npept, :]

In [7]:
distance_method = "Binomial"
ncl = 3
GMMweight = 0.5

MSC = MassSpecClustering(i, ncl, GMMweight=GMMweight, distance_method=distance_method, n_runs=1).fit(d, "NA")

run:  0
iter:  0
-23.603576191736995
iter:  1
-41.9942242904225
iter:  2
-38.747504560228364


AssertionError: EM is not improving. Prior ML: -41.9942242904225 | New ML: -38.747504560228364

In [ ]:
centers = MSC.transform(d)

In [ ]:
centers